# August 2016 Chicago Energy Benchmarking 
------
#### Data Cleaning and Visualization Workbook

In [2]:
import csv
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import matplotlib
import pandas as pd
% matplotlib inline

In [3]:
# Reading in the Data
df = pd.read_csv("Final Data.csv", encoding='iso-8859-1')

# Limiting to rows that we want to include (excluding outliers)
df = df[df['Include in Data Analysis'] == 1]

### Analyses 5 & 6
WN Site and Source EUI Distributions by Property Type
- Histograms and Boxplots

In [8]:
# Casting Source and Site EUI columns to numeric type
df['Weather Normalized Source EUI (kBtu/ft2)'] = (
    pd.to_numeric(
        df['Weather Normalized Source EUI (kBtu/ft2)'], 
            errors='coerce'))

df['Weather Normalized Site EUI (kBtu/ft2)'] = (
    pd.to_numeric(
        df['Weather Normalized Site EUI (kBtu/ft2)'], 
            errors='coerce'))

# Creating a series for each column
sources = df['Weather Normalized Source EUI (kBtu/ft2)']
sites = df['Weather Normalized Site EUI (kBtu/ft2)']

#### 5 - Source EUI Histograms and Box Plots

In [10]:
# Getting rid of outliers outside of 3 standard deviations
# and Plotting the Histogram

# fig = sources[~((sources-sources.mean()).abs()>3*sources.std())].hist(
#     bins=100, 
#     figsize=(8, 6),
#     color="steelblue")
# plt.suptitle("Weather Normalized Source EUI Histogram", size=18)
# plt.xlabel("WN Source EUI", size=14)
# plt.ylabel("Number of Buildings", size=14)
# plt.savefig("Data and Visualizations/5a - WN Source EUI Histogram.png")

In [11]:
# Plotting the Box and Whisker for Source EUI

# bp = df.boxplot(column=['Weather Normalized Source EUI (kBtu/ft2)'], 
#                 by=['Primary Property Type Category - Reduced'],
#                 figsize=(8, 6))
# plt.suptitle("WN Source EUI Boxplots Grouped by Primary Property Type", size=16)
# plt.title("")
# plt.ylim(0,1200)
# plt.xticks(rotation=50)
# plt.ylabel("Weather Normalized Source EUI", size=14)
# plt.xlabel("Primary Property Type", size=14)
# plt.savefig("Data and Visualizations/5b - WN Source EUI Boxplots.png")

#### 6 - Site EUI Histograms and Boxplots

In [12]:
# Getting rid of outliers outside of 3 standard deviations
# and Plotting the Histogram

# fig = sites[~((sites-sites.mean()).abs()>3*sites.std())].hist(bins=100, 
#                                                           figsize=(8, 6),
#                                                           color="steelblue")
# plt.suptitle("Weather Normalized Site EUI Histogram", size=18)
# plt.xlabel("WN Site EUI", size=14)
# plt.ylabel("Number of Buildings", size=14)
# plt.savefig("Data and Visualizations/6a - WN Site EUI Histogram.png")

In [13]:
# Plotting the Box and Whisker for Site EUI

# bp = df.boxplot(column=['Weather Normalized Site EUI (kBtu/ft2)'], 
#                 by=['Primary Property Type Category - Reduced'],
#                 figsize=(8, 6))
# plt.suptitle("WN Site EUI Boxplots Grouped by Primary Property Type", size=16)
# plt.title("")
# plt.ylim(0,800)
# plt.xticks(rotation=50)
# plt.ylabel("Weather Normalized Site EUI", size=14)
# plt.xlabel("Primary Property Type", size=14)
# plt.savefig("Data and Visualizations/6b - WN Source EUI Boxplots.png")

#### 5 - WN source EUI distribution with Custom Bins

In [15]:
# Binning
bins = list(range(0, 701, 50))
bins.append(9000)
labels = ['0-50', '51-100', '101-150', '151-200', '201-250', 
          '251-300', '301-350', '351-400', '401-450', '451-500', 
          '501-550', '551-600','601-650', '651-700', '700+']
source_eui_binned = pd.cut(sources, bins, labels=labels)
column_label = 'source_eui_binned'
df[column_label] = source_eui_binned

# Grouped by Source EUI bin only
source_eui_counts = df[[column_label, 'Include in Data Analysis']].groupby(
    'source_eui_binned').count()

# Writing to csv & plotting
source_eui_counts.to_csv('Data and Visualizations/5 - WN source EUI distribution.csv')

# source_eui_counts.plot.bar(legend=False, 
#                            title="Source EUI Histogram", 
#                            color="steelblue",
#                            width=1,
#                            rot=60)

# Grouped by Source EUI bin and Property Type, writing to csv
source_eui_binned_by_proptype = df[[column_label, 
    'Primary Property Type Category - Reduced', 
    'Include in Data Analysis']
  ].groupby(['Primary Property Type Category - Reduced', 
             column_label]).count()
source_eui_binned_by_proptype.to_csv('Data and Visualizations/5 - WN source EUI distribution by prop type.csv')

#### 6 - WN site EUI distribution with Custom Bins

In [14]:
# Binning
bins = list(range(0, 401, 25))
bins.append(2000)
column_label = 'site_eui_binned'
labels = ['0-25', '26-50', '51-75', '75-100', '101-125', '125-150',
          '151-175', '176-200', '201-225', '226-250', '251-275', 
          '276-300', '301-325', '326-350', '351-375', '375-400',
         '401+']
site_eui_binned = pd.cut(sites, bins, labels=labels)
df[column_label] = site_eui_binned

# Grouped by Source EUI bin only
site_eui_counts = df[[column_label, 'Include in Data Analysis']
                    ].groupby(column_label).count()

# Writing to csv, and plotting

site_eui_counts.to_csv('Data and Visualizations/6 - WN site EUI distribution.csv')
# site_eui_counts.plot.bar(legend=False, 
#                          title="Site EUI Histogram", 
#                          color="steelblue",
#                          width=1,
#                          rot=60)

# Grouped by Site EUI bin and Property Type, writing to csv
df[[column_label, 
    'Primary Property Type Category - Reduced', 
    'Include in Data Analysis']
  ].groupby(
        ['Primary Property Type Category - Reduced', 'site_eui_binned']
        ).count().to_csv('Data and Visualizations/6 - WN site EUI distribution by prop type.csv')

### Analysis 8
Number of Buildings by Decade by Property Type
 - Barplot

In [11]:
# Aggregating decades 1880 and earlier
df['Decade of Construction'].replace(
    to_replace=['1880', '1870', '1860', '1840'], 
    value='Before 1890', inplace=True)

# Grouping by Property Type then Decade
propType_decade = df.groupby(
    ['Primary Property Type Category - Reduced', 
     'Decade of Construction']).count()

# Not Grouped by Property Type
decade = df.groupby('Decade of Construction').count()

# Re-ordering indexes numerically
propType_decade.sort_index()
index = decade.index.tolist()
reindex = index[-1:] + index[:-1]
decade = decade.reindex(reindex)

# Writing to CSV
propType_decade.iloc[:, 0:1].to_csv(
    'Data and Visualizations/8 - Decade Built by Property Type.csv')
decade.to_csv("Data and Visualizations/8 - Decade Built.csv")

In [16]:
# Plotting Number of Buildings Built by Decade

# decade['Include in Data Analysis'].plot(
#     kind='bar', 
#     title ='Building Counts by Decade of Construction', 
#     color="steelblue", figsize=(8, 6))
# plt.xticks(rotation=50)

### Analysis 7
Median Source EUI, Year Built, and GFA by Property Type

In [13]:
# 7 Part A Median WN Source EUI by Property Type
median_proptype_sourceEUI = df[
    ['Weather Normalized Source EUI (kBtu/ft2)', 
     'Primary Property Type Category - Reduced']
        ].groupby('Primary Property Type Category - Reduced').median()
median_proptype_sourceEUI.to_csv('Data and Visualizations/7a-Median_Source_EUI_by_PropertyType.csv')

In [14]:
# 7 Part B Median Year Built by Property Type
median_proptype_yearBuilt = df[
    ['Year Built', 
     'Primary Property Type Category - Reduced']
        ].groupby('Primary Property Type Category - Reduced').median()
median_proptype_yearBuilt.to_csv('Data and Visualizations/7b-Median_YearBuilt_by_PropertyType.csv')

In [15]:
# 7 Part C Total Floor Area by Property Type 
df['Property GFA - EPA Calculated (Buildings) (ft2)']= pd.to_numeric(
    df['Property GFA - EPA Calculated (Buildings) (ft2)'])
total_GFA_by_proptype = df[
    ['Property GFA - EPA Calculated (Buildings) (ft2)', 
     'Primary Property Type Category - Reduced']
        ].groupby('Primary Property Type Category - Reduced').sum()
total_GFA_by_proptype.to_csv("Data and Visualizations/7c-Total_GFA_by_PropertyType.csv")

### Analysis 10
Median Site and Source EUI by Decade

In [16]:
Median_Site_Source_EUI_by_Decade = df[
    ['Decade of Construction', 
     'Weather Normalized Source EUI (kBtu/ft2)', 
     'Weather Normalized Site EUI (kBtu/ft2)']
        ].groupby('Decade of Construction').median()
Median_Site_Source_EUI_by_Decade.to_csv('Data and Visualizations/10-Median_Site_and_Source_EUI_by_Decade.csv')

In [17]:
GFA_bins = list(range(0, 4000000, 250000))
GFA_bins.append(10000000)
df['Property GFA - EPA Calculated (Buildings) (ft2)'].describe()

count       2259.000000
mean      263503.206817
std       424274.589018
min        50000.000000
25%        77442.500000
50%       131462.000000
75%       277948.000000
max      9245333.000000
Name: Property GFA - EPA Calculated (Buildings) (ft2), dtype: float64

### Analysis 12
Median EUI vs Size (binned by GFA)

In [18]:
#12 - Median EUI vs Size (binned by GFA)
df['GFA_bins'] = pd.cut(df[
        'Property GFA - EPA Calculated (Buildings) (ft2)'], GFA_bins)
source_and_site_EUI_by_propsize = df[[
        'GFA_bins', 'Weather Normalized Source EUI (kBtu/ft2)', 
        'Weather Normalized Site EUI (kBtu/ft2)']].groupby('GFA_bins').median()
source_and_site_EUI_by_propsize.to_csv("Data and Visualizations/12 - Median EUI vs Size.csv")

### Analysis 13 
Site and Source EUI vs Size (not-binned)


In [19]:
site_source_EU_vs_size = df[[
        'Weather Normalized Source EUI (kBtu/ft2)', 
        'Weather Normalized Site EUI (kBtu/ft2)', 
        'Property GFA - EPA Calculated (Buildings) (ft2)']]
site_source_EU_vs_size.to_csv("Data and Visualizations/13 - EUI vs Size scatter plot.csv")

### Analysis 14
Median site and source EUI by secondary space use

In [20]:
site_source_EU_vs_secondary = df[[
        'Weather Normalized Source EUI (kBtu/ft2)', 
        'Weather Normalized Site EUI (kBtu/ft2)', 
        '% of Floor Area Occupied by Additional Space Use Types (Percentage)']]

site_source_EU_vs_secondary.to_csv("Data and Visualizations/14 - EUI vs secondary space use.csv")

### Analyses 16 & 17
Weather Normalized Site and Source EUI 

In [17]:
# df['Weather Normalized Site EUI (kBtu/ft2)'].plot.box(whis=100, figsize=(8, 6))
# plt.title('Weather Normalized Site EUI Boxplot')

In [18]:
# df['Weather Normalized Source EUI (kBtu/ft2)'].plot.box(whis=100, figsize=(8, 6))
# plt.title('Weather Normalized Source EUI Boxplot')

In [19]:
# df['Weather Normalized Source EUI (kBtu/ft2)'].describe()